In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Ctrl+Enter) will list all files under the input directory

import os
        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import os #Importing the OS library to perform tasks such as shifting of images to folder etc.
import csv #Importing the csv library to use csv.reader function to read a file
data = {}  #data with key as the class of the image and the values as the array with all the images 
#belonging to that class
with open('/kaggle/input/state-farm-distracted-driver-detection/driver_imgs_list.csv') as f:
    reader = csv.reader(f)
    next(reader) #Skipping the header row
    for row in f:
        row = row.split(",")
        key = row[1]
        if(key in data):
            data[key].append(row[2])
        else:
            data[key]=[row[2]]
#Creating the data dictionary as the key and values specified above

In [6]:
class_list = list(data.keys()) #The various class in the dataset

In [7]:
import os
os.mkdir('/kaggle/working/master_data') #Creating directory to store the master-data
os.mkdir('/kaggle/working/master_data/training') #Creating directory to store the training data
os.mkdir('/kaggle/working/master_data/validation') #Creating directory to store the validation data

In [8]:
for class_ in class_list:
    os.mkdir('/kaggle/working/master_data/training/'+class_) #Creating different directory corresponding to each class in the
    #training and validation directory so as to store images belonging to each class in different folders
    #This would enable us to use the ImageDataGenerator library
    os.mkdir('/kaggle/working/master_data/validation/'+class_)
    #Performing the same task for the validation directory

In [9]:
from shutil import copyfile #Copyfile lib to copy file from source to destination directory
split_size = 0.8 #The ratio in which the validation and training data will be split

for class_, images in data.items():
    train_size = int(split_size * len(images))
    train_images = images[:train_size] #The training images
    test_images = images[train_size:] #The testing images
    #The for loops are used for storing the data in the corresponding class from the train directory
    for image in train_images:
        image = image.split("\n") 
        source = os.path.join('/kaggle/input/state-farm-distracted-driver-detection/imgs/train',class_,image[0]) #source directory
        dest = os.path.join('/kaggle/working/master_data/training',class_,image[0]) #destination directory
        copyfile(source,dest) # copying the file from source to dest
    ##The for loops are used for storing the data in the corresponding class from the train directory
    for image in test_images:
        image = image.split("\n")
        source = os.path.join('/kaggle/input/state-farm-distracted-driver-detection/imgs/train',class_,image[0]) #Source Directory
        dest = os.path.join('/kaggle/working/master_data/validation',class_,image[0]) #Destination Directory
        copyfile(source,dest) #Copying file from source to destination
    

In [10]:
import tensorflow as tf #Importing tensorflow library
from tensorflow.keras.preprocessing.image import ImageDataGenerator #Importing ImageDataGenerator library to easily manage 
#and create the training and validation dataset
from tensorflow.keras.optimizers import Adam
#Adam Optimizer
from tensorflow.keras.callbacks import EarlyStopping
#Callbacks to stop the epochs if the change in accuracy is not significant
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Dropout
#Importing various layers

In [16]:
#Creating the model by adding various layers such as the CONV2D, MaxPooling2D, dropout layer, dense layer etc.
model = tf.keras.models.Sequential([
    Conv2D(32,(2,2),activation='relu',input_shape = (128,128,3),kernel_initializer='glorot_normal',padding='same'),
    MaxPooling2D(pool_size = (2,2),),
    Conv2D(64,(2,2),activation = 'relu',padding='same',kernel_initializer='glorot_normal'),
    MaxPooling2D(pool_size = (2,2)),
    Conv2D(128,(2,2),activation = 'relu',padding='same',kernel_initializer='glorot_normal'),
    MaxPooling2D(pool_size = (2,2)),
    Dropout(0.5),
    Flatten(),
    Dense(500,activation = 'relu',kernel_initializer='glorot_normal'),
    Dropout(0.5),
    Dense(10,activation = 'softmax',kernel_initializer='glorot_normal')
])
#Compiling the model with the loss, optimizer and metrics used for testing the model
model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy',
             metrics = ['accuracy'])
#Checking the model summary if correct or not
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 128, 128, 32)      416       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 64, 64)        8256      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 128)       32896     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 128)      

In [17]:
train_dir = '/kaggle/working/master_data/training' #Directory containing the training data
test_dir = '/kaggle/working/master_data/validation' #Directory containing the testing data

train_data_gen = ImageDataGenerator(1.0/255) #Image Data Generator for the trainging dataset
train_generator = train_data_gen.flow_from_directory(
    train_dir , target_size = (128,128), class_mode = 'categorical',
    batch_size = 128
) #Creating the training_generator with specified parameters
test_data_gen = ImageDataGenerator(1.0/255) #Image Data Generator for the validation dataset
test_generator = test_data_gen.flow_from_directory(
    test_dir , target_size = (128,128), class_mode = 'categorical',
    batch_size = 128
) #Creating the validation_generator with specified parameters


Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


In [19]:
es = EarlyStopping(monitor = 'accuracy',patience = 3, min_delta = 0.01)
#Defining the callback to stop the model with accuracy as the metrics and min_delta 1% and patience 3 epochs

In [20]:
model.fit_generator(train_generator,epochs = 20, verbose =1, validation_data=test_generator,callbacks=[es])

Epoch 1/20
141/141 [==============================] - 290s 2s/step - loss: 67.9278 - accuracy: 0.3987 - val_loss: 5.3390 - val_accuracy: 0.1486
Epoch 2/20
141/141 [==============================] - 289s 2s/step - loss: 0.8968 - accuracy: 0.8457 - val_loss: 9.8045 - val_accuracy: 0.1539
Epoch 3/20
141/141 [==============================] - 303s 2s/step - loss: 0.4198 - accuracy: 0.9213 - val_loss: 7.7563 - val_accuracy: 0.1904
Epoch 4/20
141/141 [==============================] - 288s 2s/step - loss: 0.3425 - accuracy: 0.9368 - val_loss: 7.2158 - val_accuracy: 0.2080
Epoch 5/20
141/141 [==============================] - 305s 2s/step - loss: 0.1958 - accuracy: 0.9564 - val_loss: 10.9925 - val_accuracy: 0.2336
Epoch 6/20
141/141 [==============================] - 288s 2s/step - loss: 0.4608 - accuracy: 0.9414 - val_loss: 12.1593 - val_accuracy: 0.2285
Epoch 7/20
141/141 [==============================] - 308s 2s/step - loss: 0.1280 - accuracy: 0.9747 - val_loss: 12.8780 - val_accuracy: 0.

In [ ]:
import glob
from skimage import io 
from skimage.transform import resize
size = (128,128)
image_dir = '/kaggle/input/state-farm-distracted-driver-detection/imgs/test'
image_list = glob.glob(image_dir+'/*')
x_test= [] #Creating the testing dataset by iterating the test directory and resizing the images according to the expected data.
for i,imgfile in enumerate(image_list):
    if(i%10000==0):
        print(i)
    img = io.imread(imgfile)
    img = resize(img,size)
    x_test.append(img)
    #appending the current image to x_test


0


In [ ]:
#Prediciting the result and printing it
x_test = np.array(x_test) 
print(x_test.shape)
y_pred = model.predict(x_test)
print(y_pred)